In [56]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import warnings

In [57]:
warnings.filterwarnings('ignore')
pd.set_option('precision', 2)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

数据引入

In [58]:
# 引入股票数据
imsf=pd.read_csv(r"C:\Users\86196\Desktop\资产定价复现\data\imsf.csv")
imsf['DATE']=pd.to_datetime(imsf['DATE'])

In [59]:
# 引入市场信息,来源于https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
mkt_df=pd.read_csv(r"C:\Users\86196\Desktop\资产定价复现\data\F-F_Research_Data_Factors.CSV",nrows=1161)
# 处理时序，并转为时间序列
mkt_df=mkt_df.rename(columns={'Unnamed: 0':'Date'})
mkt_df['Date']=pd.to_datetime(mkt_df['Date'],format='%Y%m') 
mkt_df['Date']=mkt_df['Date']+pd.tseries.offsets.MonthEnd(0)
mkt_df=mkt_df.set_index('Date')

辅助工作

In [60]:
# 得到时间分组
date_group=imsf.groupby('DATE')

In [61]:
#newey-west检验量
def newey_west(timeval,lag=-1):
    T=timeval.shape[0]
    L=np.floor(4*(T/100)**(2/9))
    L=int(L)
    if lag>=0:
        L=lag
    e=timeval-timeval.mean()
    wl=1-np.arange(1,L+1)/(L+1)
    p_1=(e**2).sum()
    p_2=0
    for i in range(1,L+1):
        p_2+=2*(e*e.shift(i)).sum()*wl[i-1]
    ste=np.sqrt(p_1+p_2)/T
    t=timeval.mean()/ste
    return t.item()

表-9.1

In [62]:
# 创造表-9.1
rowname=['MktCap','Size','MktCap_CPI','Size_CPI','MktCap_FF','Size_FF','MktCap_FF_CPI','Size_FF_CPI']
colname=['Mean','SD','Skew','Kurt','Min','5%','25%','Median','75%','95%','Max','n']
statsdf=pd.DataFrame(np.zeros((8,12)),index=rowname,columns=colname)

In [63]:
# 用于月度计算统计特征的函数
def monthly_feature(x,rowname):
    mfm=np.zeros((8,12))
    for i,name in enumerate(rowname):
        data=x[name].dropna()
        mfm[i]=np.mean(data),np.std(data),stats.skew(data),stats.kurtosis(data),np.min(data),np.percentile(data,5),np.percentile(data,25),np.median(data),np.percentile(data,75),np.percentile(data,95),np.max(data),len(data)
    return mfm

In [64]:
# 得到月度特征
statsdf.loc[:]=date_group.apply(monthly_feature,rowname=rowname).values.mean(axis=0)

In [65]:
# 显示表-9.1
statsdf

,Mean,SD,Skew,Kurt,Min,5%,25%,Median,75%,95%,Max,n
MktCap,2181.53,10325.95,16.51,409.13,0.72,8.91,52.85,227.64,969.71,8264.59,283959.01,4739.17
Size,4.81,2.00,0.30,-0.15,-0.87,1.75,3.36,4.69,6.13,8.32,11.96,4739.17
MktCap_CPI,3290.48,15794.70,16.51,409.13,1.47,15.54,84.07,345.17,1460.24,12431.48,445876.32,4739.17
Size_CPI,5.61,2.00,0.30,-0.15,-0.07,2.55,4.16,5.49,6.93,9.11,12.76,4739.17
MktCap_FF,2113.97,9888.08,16.45,404.76,0.82,9.65,54.38,227.31,950.45,8058.14,268219.41,4723.49
Size_FF,4.82,1.96,0.34,-0.13,-0.64,1.84,3.40,4.69,6.11,8.29,11.93,4723.49
MktCap_FF_CPI,3189.09,15180.40,16.45,404.76,1.65,16.75,86.24,343.75,1428.00,12101.34,425068.58,4723.49
Size_FF_CPI,5.62,1.96,0.34,-0.13,0.16,2.64,4.20,5.49,6.91,9.09,12.73,4723.49


表-9.2

In [66]:
# 创建表-9.2
rowname=['MktCap','Size','MktCap_FF','Size_FF','beta']
colname=rowname
corrdf=pd.DataFrame(np.zeros((5,5))*np.nan,index=rowname,columns=colname)

In [67]:
# 计算月度相关系数均值
for i in range(5):
    for j in range(i):
        corrdf.iloc[i,j]=date_group.apply(lambda x:x[rowname[i]].corr(x[rowname[j]])).mean()
        corrdf.iloc[j,i]=date_group.apply(lambda x:stats.spearmanr(x[rowname[i]], x[rowname[j]])[0]).mean()

In [68]:
# 显示表-9.2
corrdf

,MktCap,Size,MktCap_FF,Size_FF,beta
MktCap,NaN,1.00,0.99,0.99,-0.28
Size,0.43,NaN,0.99,0.99,-0.28
MktCap_FF,0.99,0.43,NaN,1.00,-0.28
Size_FF,0.43,0.98,0.43,NaN,-0.28
beta,-0.04,-0.25,-0.04,-0.26,NaN


表-9.3

In [69]:
# 创造表-9.3
rowname=[1,3,6,12,24,36,48,60,120]
colname=['MktCap','Size','MktCap_FF','Size_FF']
persistdf=pd.DataFrame(np.zeros((9,4)),index=rowname,columns=colname)

In [70]:
# 创造辅助数据表
asist_0=imsf[['PERMNO','DATE']+colname].dropna(subset=colname).copy()

In [71]:
# 计算不同时间间隔下的相关系数均值
for gap in rowname:
    asist=asist_0.copy()
    asist['DATE']+=pd.offsets.MonthEnd(gap)
    persist=asist_0.merge(asist,how='left',on=['PERMNO','DATE'],suffixes=('','_gap')).dropna()
    persist=persist.groupby('DATE')
    persistdf.loc[gap]=persist.apply(lambda x:np.array([x[col].corr(x[col+'_gap']) for col in colname])).mean(axis=0)

In [72]:
# 显示表-9.3
persistdf

,MktCap,Size,MktCap_FF,Size_FF
1,1.00,1.00,1.00,1.00
3,0.99,0.99,0.99,0.99
6,0.99,0.98,0.99,0.98
12,0.98,0.97,0.98,0.97
24,0.95,0.94,0.95,0.94
36,0.93,0.92,0.93,0.92
48,0.91,0.91,0.91,0.91
60,0.89,0.89,0.89,0.89
120,0.79,0.84,0.79,0.84


表-9.4

In [73]:
# 创造panel-A
rowname=['MktCap','MktCap_CPI','%MktCap','%NYSE','n','beta','MktCap_FF','MktCap_FF_CPI','%MktCap_FF','%NYSE','n','beta']
colname=['1','2','3','4','5','6','7','8','9','10']
UPA_df_A=pd.DataFrame(np.zeros([12,10]),index=rowname,columns=colname)

In [74]:
# 辅助函数,用于计算月度特征
def get_A(x,col):
    # 数据容器
    val_matrix=np.zeros([6,10])
    # 计算断点
    breakpoint=x[x.EXCHCD==1][col].quantile(np.arange(0,1+1/10,1/10))
    # 获取特征
    for i in range(10):
        set2cal=x[(x[col]>=breakpoint.iloc[i])&(x[col]<=breakpoint.iloc[i+1])]
        val_matrix[:,i]=set2cal[col].mean(),set2cal[col+'_CPI'].mean(),set2cal[col].sum()/x[col].sum()*100,len(set2cal[set2cal.EXCHCD==1])/len(set2cal)*100,len(set2cal),set2cal['beta'].mean()
    return val_matrix

In [75]:
# 获得月度特征均值
MktCap_val=date_group.apply(lambda x:get_A(x,'MktCap')).mean(axis=0)
MktCap_FF_val=date_group.apply(lambda x:get_A(x,'MktCap_FF')).mean(axis=0)
# 填充数据
UPA_df_A[:6]=MktCap_val
UPA_df_A[6:]=MktCap_FF_val

In [76]:
# 显示panel-A
UPA_df_A

,1,2,3,4,5,6,7,8,9,10
MktCap,51.48,190.67,348.45,559.36,850.65,1266.03,1915.75,3200.43,6386.78,30840.51
MktCap_CPI,86.90,312.16,561.58,894.53,1361.18,2035.13,3098.26,5164.85,10146.44,48478.29
%MktCap,1.45,1.45,1.72,2.21,2.84,3.61,5.04,7.72,13.58,58.95
%NYSE,8.54,27.49,39.34,48.13,57.08,67.18,73.66,79.50,85.21,89.65
n,1987.16,589.46,395.50,319.22,267.40,225.24,204.55,189.96,177.30,168.28
beta,-0.74,-1.00,-1.06,-1.07,-1.08,-1.08,-1.08,-1.08,-1.05,-1.03
MktCap_FF,53.43,195.23,350.13,556.41,839.23,1243.24,1874.13,3121.64,6225.75,30073.02
MktCap_FF_CPI,89.79,318.04,562.37,887.14,1339.09,1993.73,3023.33,5031.77,9878.71,47232.91
%MktCap_FF,1.55,1.49,1.74,2.22,2.83,3.60,5.03,7.74,13.61,58.94
%NYSE,8.45,27.89,39.90,48.75,57.75,67.87,74.03,79.61,85.26,89.93


In [77]:
# 创造panel-B、C
rowname=['ExRET','ExRET-t','alpha','alpha-t','ExRET_FF','ExRET_FF-t','alpha_FF','alpha_FF-t']
colname=['1','2','3','4','5','6','7','8','9','10','10-1']
UPA_df_B=pd.DataFrame(np.zeros([8,11]),index=rowname,columns=colname)
UPA_df_C=UPA_df_B.copy()

In [78]:
# 辅助函数,用于计算组合月度收益与alpha
def get_UPA_seq(x,mktval):
    col=['MktCap','MktCap_FF']
    # 数据容器
    val_matrix=np.zeros([11,8])
    # 计算断点
    div=np.arange(0,1+1/10,1/10)
    breakpoint=x[x.EXCHCD==1][col[0]].quantile(div)
    breakpoint_FF=x[x.EXCHCD==1][col[1]].quantile(div)
    # 为了保证全部划分，需要将上下界设为全局最大最小值，而不是仅考虑x.EXCHCD==1的情况
    breakpoint[0],breakpoint[-1]=x[col[0]].min(),x[col[0]].max()
    breakpoint_FF[0],breakpoint_FF[-1]=x[col[1]].min(),x[col[1]].max()
    # 获取特征
    for i in range(10):
        # 分组
        up4c=x[(x[col[0]]>=breakpoint.iloc[i])&(x[col[0]]<=breakpoint.iloc[i+1])]
        up4c_FF=x[(x[col[1]]>=breakpoint_FF.iloc[i])&(x[col[1]]<=breakpoint_FF.iloc[i+1])]
        # 计算市值加权权重
        weight=up4c[col[0]].values/up4c[col[0]].sum()
        weight_FF=up4c_FF[col[1]].values/up4c_FF[col[1]].sum()
        # 计算t+1期的收益及beta
        val_matrix[i,:4]=up4c['ExRET_t+1'].mean(),up4c['beta'].mean(),up4c_FF['ExRET_t+1'].mean(),up4c_FF['beta'].mean()
        val_matrix[i,4:]=up4c['ExRET_t+1'].values.dot(weight),up4c['beta'].values.dot(weight),up4c_FF['ExRET_t+1'].values.dot(weight_FF),up4c_FF['beta'].values.dot(weight_FF)
    # 计算alpha
    val_matrix[:,[1,3,5,7]]=val_matrix[:,[0,2,4,6]]-val_matrix[:,[1,3,5,7]]*mktval
    # 计算10-1组
    val_matrix[10]=val_matrix[9]-val_matrix[0]
    return val_matrix

In [79]:
# 获得组合收益时间序列
UPA_seq=date_group.apply(lambda x:get_UPA_seq(x,mkt_df.loc[x.iloc[0].DATE,'Mkt-RF'].mean()))

In [80]:
# 取月度均值填充收益与alpha
UPA_df_B.loc[['ExRET','alpha','ExRET_FF','alpha_FF']]=UPA_seq.mean(axis=0)[:,:4].T
UPA_df_C.loc[['ExRET','alpha','ExRET_FF','alpha_FF']]=UPA_seq.mean(axis=0)[:,4:].T

In [81]:
# 计算t统计量
UPA_nm=np.stack(UPA_seq.values,axis=0)
for i,col in enumerate(colname):
    seq4T=pd.DataFrame(UPA_nm[:,i,:])
    UPA_df_B.loc['ExRET-t',col]=newey_west(seq4T.loc[:,0])
    UPA_df_B.loc['alpha-t',col]=newey_west(seq4T.loc[:,1])
    UPA_df_B.loc['ExRET_FF-t',col]=newey_west(seq4T.loc[:,2])
    UPA_df_B.loc['alpha_FF-t',col]=newey_west(seq4T.loc[:,3])
    UPA_df_C.loc['ExRET-t',col]=newey_west(seq4T.loc[:,4])
    UPA_df_C.loc['alpha-t',col]=newey_west(seq4T.loc[:,5])
    UPA_df_C.loc['ExRET_FF-t',col]=newey_west(seq4T.loc[:,6])
    UPA_df_C.loc['alpha_FF-t',col]=newey_west(seq4T.loc[:,7])

In [82]:
# 显示panel-B（等权重）
UPA_df_B

,1,2,3,4,5,6,7,8,9,10,10-1
ExRET,1.05,0.75,0.79,0.81,0.80,0.77,0.73,0.74,0.69,0.60,-0.46
ExRET-t,3.26,2.74,3.06,3.28,3.35,3.34,3.28,3.44,3.39,3.19,-1.98
alpha,1.49,1.39,1.48,1.50,1.51,1.49,1.46,1.46,1.39,1.28,-0.22
alpha-t,3.41,3.20,3.44,3.54,3.63,3.60,3.58,3.63,3.60,3.51,-0.95
ExRET_FF,1.04,0.75,0.79,0.77,0.81,0.75,0.77,0.74,0.72,0.60,-0.43
ExRET_FF-t,3.31,2.69,3.05,3.10,3.28,3.17,3.37,3.30,3.35,3.12,-1.98
alpha_FF,1.50,1.40,1.48,1.46,1.50,1.45,1.46,1.42,1.38,1.24,-0.25
alpha_FF-t,3.49,3.18,3.44,3.42,3.51,3.46,3.53,3.46,3.47,3.34,-1.13


In [83]:
# 显示panel-C（市值加权）
UPA_df_C

,1,2,3,4,5,6,7,8,9,10,10-1
ExRET,0.73,0.74,0.80,0.81,0.80,0.76,0.74,0.74,0.69,0.58,-0.15
ExRET-t,2.44,2.74,3.08,3.28,3.36,3.32,3.30,3.45,3.38,3.18,-0.73
alpha,1.25,1.39,1.49,1.50,1.52,1.48,1.46,1.46,1.39,1.23,-0.02
alpha-t,2.88,3.20,3.45,3.54,3.64,3.59,3.59,3.64,3.62,3.52,-0.08
ExRET_FF,0.80,0.73,0.79,0.78,0.82,0.75,0.78,0.73,0.71,0.58,-0.21
ExRET_FF-t,2.70,2.64,3.06,3.11,3.31,3.18,3.38,3.30,3.32,3.14,-1.02
alpha_FF,1.32,1.39,1.49,1.46,1.51,1.45,1.47,1.41,1.37,1.20,-0.12
alpha_FF-t,3.10,3.15,3.45,3.42,3.53,3.47,3.54,3.46,3.47,3.35,-0.57


表-9.6

In [84]:
# 创造panel-A/B
rowname=['MktCap 1','MktCap 2','MktCap 3','MktCap 4','MktCap 5','MktCap 5-1','ExRET-t','alpha','alpha-t']
colname=['beta 1','beta 2','beta 3','beta 4','beta 5','beta avg']
BPA_df_A=pd.DataFrame(np.zeros([9,6]),index=rowname,columns=colname)
BPA_df_B=BPA_df_A.copy()

In [85]:
# 辅助函数,用于计算组合月度收益与alpha
# 此处为条件排序，即按照beta排序后再按照市值排序
def get_BPA_seq(x,mktval):
    col='MktCap'
    # 数据容器
    val_matrix_ew=np.zeros([7,6])
    val_matrix_vw=val_matrix_ew.copy()
    # 基于NYSE数据计算beta断点
    div=np.arange(0,1+1/5,1/5)
    beta_breakpoint=x[x.EXCHCD==1]['beta'].quantile(div)
    beta_breakpoint[0],beta_breakpoint[-1]=x['beta'].min(),x['beta'].max()
    for i in range(5):
        y=x[(x['beta']>=beta_breakpoint.iloc[i])&(x['beta']<=beta_breakpoint.iloc[i+1])]
        # 基于beta分组NYSE数据计算市值断点
        breakpoint=y[y.EXCHCD==1][col].quantile(div)
        breakpoint[0],breakpoint[-1]=y[col].min(),y[col].max()
        for j in range(5):
            # 分组
            bp4c=y[(y[col]>=breakpoint.iloc[j])&(y[col]<=breakpoint.iloc[j+1])]
            # 计算权重
            weight=bp4c[col].values/bp4c[col].sum()
            # 计算t+1期的收益
            val_matrix_ew[j,i]=bp4c['ExRET_t+1'].mean()
            val_matrix_vw[j,i]=bp4c['ExRET_t+1'].values.dot(weight)
            # 记录市值分组两端的beta值，用于多空组合beta值计算
            if j==0:
                val_matrix_ew[-1,i]=-bp4c['beta'].mean()
                val_matrix_vw[-1,i]=-bp4c['beta'].values.dot(weight)
            if j==5:
                val_matrix_ew[-1,i]+=bp4c['beta'].mean()
                val_matrix_vw[-1,i]+=bp4c['beta'].values.dot(weight)
    # 计算5-1组收益
    val_matrix_ew[-2]=val_matrix_ew[-3]-val_matrix_ew[0]
    val_matrix_vw[-2]=val_matrix_vw[-3]-val_matrix_vw[0]
    # 计算alpha
    val_matrix_ew[-1]=val_matrix_ew[-2]-val_matrix_ew[-1]*mktval
    val_matrix_vw[-1]=val_matrix_vw[-2]-val_matrix_vw[-1]*mktval
    # 计算beta avg组收益
    val_matrix_ew[:,-1]=val_matrix_ew[:,:-1].mean(axis=1)
    val_matrix_vw[:,-1]=val_matrix_vw[:,:-1].mean(axis=1)
    # 合并返回val_matrix_ew与val_matrix_vw
    return np.concatenate([val_matrix_ew,val_matrix_vw],axis=0)

In [86]:
# 获得组合收益时间序列
BPA_seq=date_group.apply(lambda x:get_BPA_seq(x,mkt_df.loc[x.iloc[0].DATE,'Mkt-RF'].mean()))

In [87]:
# 取月度均值填充收益与alpha
BPA_df_A.loc[['MktCap 1','MktCap 2','MktCap 3','MktCap 4','MktCap 5','MktCap 5-1','alpha']]=BPA_seq.mean(axis=0)[:7]
BPA_df_B.loc[['MktCap 1','MktCap 2','MktCap 3','MktCap 4','MktCap 5','MktCap 5-1','alpha']]=BPA_seq.mean(axis=0)[7:]

In [88]:
# 计算t统计量
BPA_nm=np.stack(BPA_seq.values,axis=0)
for i,col in enumerate(colname):
    seq4T=pd.DataFrame(BPA_nm[:,[5,6,12,13],i])
    BPA_df_A.loc['ExRET-t',col]=newey_west(seq4T.loc[:,0])
    BPA_df_A.loc['alpha-t',col]=newey_west(seq4T.loc[:,1])
    BPA_df_B.loc['ExRET-t',col]=newey_west(seq4T.loc[:,2])
    BPA_df_B.loc['alpha-t',col]=newey_west(seq4T.loc[:,3])

In [89]:
# 显示panel-A（等权重）
BPA_df_A

,beta 1,beta 2,beta 3,beta 4,beta 5,beta avg
MktCap 1,0.69,0.97,1.05,1.03,1.14,0.97
MktCap 2,0.77,0.94,0.92,0.86,0.73,0.84
MktCap 3,0.87,0.98,0.87,0.82,0.72,0.85
MktCap 4,0.73,0.90,0.80,0.81,0.65,0.78
MktCap 5,0.71,0.72,0.73,0.69,0.60,0.69
MktCap 5-1,0.02,-0.25,-0.32,-0.34,-0.53,-0.28
ExRET-t,0.07,-1.00,-1.43,-1.66,-2.61,-1.33
alpha,-1.25,-1.10,-0.96,-0.81,-0.62,-0.95
alpha-t,-2.49,-2.72,-2.80,-2.78,-2.74,-2.77


In [90]:
# 显示panel-B（市值加权）
BPA_df_B

,beta 1,beta 2,beta 3,beta 4,beta 5,beta avg
MktCap 1,0.54,0.85,0.90,0.87,0.79,0.79
MktCap 2,0.76,0.94,0.91,0.85,0.73,0.84
MktCap 3,0.87,0.98,0.87,0.80,0.71,0.85
MktCap 4,0.70,0.92,0.80,0.80,0.63,0.77
MktCap 5,0.68,0.67,0.67,0.61,0.57,0.64
MktCap 5-1,0.14,-0.18,-0.23,-0.26,-0.22,-0.15
ExRET-t,0.59,-0.85,-1.16,-1.34,-1.19,-0.79
alpha,-1.12,-1.03,-0.88,-0.73,-0.36,-0.82
alpha-t,-2.23,-2.76,-2.79,-2.65,-1.72,-2.57
